In [1]:
import numpy as np
import pylidc as pl
from unet import UNet

Using TensorFlow backend.


In [3]:
scans = pl.query(pl.Scan)
n = scans.count()

In [4]:
image_size = (256, 256)

In [5]:
def union_bboxes(anns):
    return tuple((min(d.start for d in dim), max(d.stop for d in dim)) for dim in zip(*(ann.bbox() for ann in anns)))

In [10]:
def get_data(i, samples):
    scan = scans[i]
    images = scan.load_all_dicom_images(verbose=False)
    X, y = [], []
    for nodule in scan.cluster_annotations():
        coords = union_bboxes(nodule)
        y_big = np.zeros(tuple(stop - start for start, stop in coords))
        for ann in nodule:
            b = ann.bbox()
            bb = tuple(slice(bi.start - coord[0], bi.stop - coord[0], None) for bi, coord in zip(b, coords))
            y_big[bb] = np.logical_or(y_big[bb], ann.boolean_mask())
        means = [int(sum(coord) / 2) for coord in coords]
        for zi in range(*coords[2]):
            X_big = images[zi].pixel_array
            for _ in range(samples):
                starts = [np.random.randint(max(0, mean - ims), min(512 - ims, mean)) for mean, ims in zip(means, image_size)]
                X.append(X_big[tuple(slice(start, start + ims) for start, ims in zip(starts, image_size))])
                y_blank = np.zeros(image_size)
                pos = tuple(slice(max(0, start - coord[0]), min(s, start - coord[0] + ims))
                            for start, coord, ims, s in zip(starts, coords, image_size, y_big.shape))
                y_blank[pos] = y_big[(*pos, zi - coords[2][0])]
                y.append(y_blank)
    return np.array(X), np.array(y)